In [14]:
# 从数据库导入数据dfpbroech7
import pandas as pd
from sqlalchemy import create_engine

# 根据你的实际数据库信息填写
username = "panjinhe"
password = "20020112p"
host = "localhost"
port = "5432"
database = "pbroe"

# 定义要查询的表和schema
table_name = 'pbroech6'
schema_name = 'pbroe'

# 定义日期范围
start_date = '2005-04'
end_date = '2025-03'

# 构建连接字符串
connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# 创建引擎
engine = create_engine(connection_string)

# 构建带有日期范围筛选的SQL查询语句
# 这样可以在数据库层面直接过滤，效率更高
sql_query = f"""
SELECT * FROM {schema_name}.{table_name}
WHERE "trdmnt" >= '{start_date}' AND "trdmnt" <= '{end_date}'
"""
# 使用 pd.read_sql_query 执行带有条件的查询
dfpbroech7 = pd.read_sql_query(sql_query, engine)

# print("\n数据加载成功！")
# # --- 3. 显示数据信息 ---
# print("\nDataFrame Info:")
print(dfpbroech7.info())
# print("\nDataFrame Head (first 10 rows):")
#display(dfpbroech6.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665563 entries, 0 to 665562
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   stkcd       665563 non-null  object 
 1   trdmnt      665563 non-null  object 
 2   accper      665563 non-null  object 
 3   shortname   665563 non-null  object 
 4   if_st       665563 non-null  int64  
 5   indcd1      665563 non-null  object 
 6   indnme1     665563 non-null  object 
 7   price       665563 non-null  float64
 8   market_cap  665563 non-null  float64
 9   PB          663460 non-null  float64
 10  roe_ttm     636635 non-null  float64
 11  roic        655034 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 60.9+ MB
None


In [6]:
import pandas as pd

# 假设您的主数据表 'dfpbroech7' 已经加载到内存中
# 如果没有，请先加载：
# dfpbroech7 = pd.read_csv('path_to_your_dfpbroech7_data.csv')

# --- 步骤 1: 加载持仓数据 ---
# 请确保文件路径正确
holdings_path = 'E:\\PBROE\\ch7\\PBROE_5.0_from_3.1_TS_10M_holdings.csv'
try:
    # 明确指定stkcd为字符串类型，防止读取时丢失前面的0
    holdings_df = pd.read_csv(holdings_path, dtype={'stkcd': str})
    print("持仓文件加载成功。")
except FileNotFoundError:
    print(f"错误：无法在路径 {holdings_path} 找到文件。请检查路径是否正确。")
    # 如果文件未找到，则停止执行后续代码
    # 在实际的notebook中，后续代码会因为holdings_df未定义而报错
    holdings_df = None

if holdings_df is not None:
    # --- 步骤 2: 准备用于合并的两个DataFrame ---

    # 准备 holdings_df
    # a. 为了方便，重命名中文列名
    holdings_df.rename(columns={'调入日期': 'holding_date'}, inplace=True)

    # b. 将 'holding_date' 列转换为日期时间对象
    holdings_df['holding_date'] = pd.to_datetime(holdings_df['holding_date'])

    # c. 创建一个 '年-月' 格式的合并键
    holdings_df['merge_month'] = holdings_df['holding_date'].dt.strftime('%Y-%m')

    # d. 确保stkcd是6位字符串，不足则在前面补0
    holdings_df['stkcd'] = holdings_df['stkcd'].str.zfill(6)


    # 准备 dfpbroech7
    # a. 确保 'stkcd' 是6位字符串
    dfpbroech7['stkcd'] = dfpbroech7['stkcd'].astype(str).str.zfill(6)

    # b. 将 'trdmnt' 列转换为日期时间对象
    dfpbroech7['trdmnt'] = pd.to_datetime(dfpbroech7['trdmnt'])

    # c. 创建一个 '年-月' 格式的合并键
    dfpbroech7['merge_month'] = dfpbroech7['trdmnt'].dt.strftime('%Y-%m')


    # --- 步骤 3: 执行合并 ---
    # 使用左连接（left merge），保留dfpbroech7的所有行
    # 将holdings_df的数据匹配到dfpbroech7中
    merged_df = pd.merge(dfpbroech7,
                         holdings_df,
                         on=['stkcd', 'merge_month'],
                         how='right')

    # --- 步骤 4: 验证结果 ---
    print("\n合并完成。以下是合并后数据的前5行预览：")
    print(merged_df.head())

    print("\n合并后DataFrame的基本信息：")
    merged_df.info()

    # 检查有多少行成功匹配到了持仓信息
    # 通过检查合并过来的'holding_date'列是否非空来判断
    matched_rows = merged_df['holding_date'].notna().sum()
    total_rows = len(merged_df)
    print(f"\n成功将持仓信息匹配到 {matched_rows} / {total_rows} 行。")

    # 您可以检查一下没有匹配上的情况，例如：
    # unmatched_sample = merged_df[merged_df['holding_date'].isna()].head()
    # print("\n未匹配上的数据示例：")
    # print(unmatched_sample)



持仓文件加载成功。

合并完成。以下是合并后数据的前5行预览：
    stkcd     trdmnt      accper shortname_x  if_st indcd1     indnme1_x  \
0  000027 2010-05-01  2009-12-31        深圳能源    0.0    D44   电力、热力生产和供应业   
1  000155 2010-05-01  2009-12-31        川化股份    0.0    C26  化学原料和化学制品制造业   
2  000402 2010-05-01  2009-12-31         金融街    0.0    K70          房地产业   
3  000600 2010-05-01  2009-12-31        建投能源    0.0    D44   电力、热力生产和供应业   
4  000667 2010-05-01  2009-12-31        名流置业    0.0    K70          房地产业   

   price    market_cap      PB_x  ...  merge_month  holding_date shortname_y  \
0  10.50  2.312620e+10  0.356866  ...      2010-05    2010-05-01        深圳能源   
1   6.37  2.993900e+09  0.397018  ...      2010-05    2010-05-01        川化股份   
2   8.22  2.039557e+10  0.317574  ...      2010-05    2010-05-01         金融街   
3   5.21  4.760169e+09  0.295477  ...      2010-05    2010-05-01        建投能源   
4   6.97  9.389662e+09  0.455745  ...      2010-05    2010-05-01        名流置业   

      indnme1_y    ROEttm     

In [13]:
# 用ROIC区分和回测
import pandas as pd
import numpy as np
from pathlib import Path
import traceback

# --- 步骤 0: 配置参数 ---
# 请根据您的文件存放位置修改以下路径
RETURNS_FILE = Path("E:/PBROE/data/TRDNEW_Mnth.csv")
BENCHMARK_FILE = Path("E:/PBROE/data/benchmark_indices.csv")
# 新增：定义结果输出文件夹
OUTPUT_DIR = Path("E:/PBROE/ch7/backtest_results")

# 回测参数
START_DATE = "2010-05-01"
END_DATE = "2025-04-30"
BENCHMARK_CODE = '000300'
RISK_FREE_RATE = 0.03
# 用于分组的分位数，例如0.3代表选取前30%和后30%
QUANTILE_THRESHOLD = 0.8

# 确保 'merged_df' 已经存在于您的Notebook环境中
if 'merged_df' not in locals():
    print("错误：'merged_df' 不存在。请先运行数据合并的代码。")
    # 此处可以添加加载 merged_df 的代码作为备用
    # merged_df = pd.read_csv('path_to_your_merged_df.csv', dtype={'stkcd': str})
    # merged_df['trdmnt'] = pd.to_datetime(merged_df['trdmnt'])


# --- 步骤 1: 调用回测引擎的核心函数 ---

def build_portfolio_by_quantile(
    holdings_df: pd.DataFrame,
    factor_col: str,
    quantile: float,
    select_top: bool
) -> dict:
    """
    根据指定因子和分位数构建投资组合。
    """
    df_filtered = holdings_df.dropna(subset=[factor_col]).copy()
    selections_dict = {}

    for date, group in df_filtered.groupby('trdmnt'):
        try:
            if select_top:
                # 高分组
                threshold = group[factor_col].quantile(1 - quantile)
                selected_stocks = set(group[group[factor_col] >= threshold]['stkcd'])
            else:
                # 低分组
                threshold = group[factor_col].quantile(quantile)
                selected_stocks = set(group[group[factor_col] <= threshold]['stkcd'])

            if selected_stocks:
                selections_dict[pd.to_datetime(date)] = selected_stocks
        except Exception:
            continue
    return selections_dict


def run_single_strategy_backtest(
    strategy_name: str,
    selections: dict,
    returns_df: pd.DataFrame,
    start_date: str,
    end_date: str
) -> pd.Series:
    """
    对单个策略执行向量化回测。
    """
    backtest_months = pd.to_datetime(pd.date_range(start_date, end_date, freq='MS'))
    portfolio_map = pd.Series(index=backtest_months, dtype='object')
    rebalance_dates = sorted(selections.keys())

    if rebalance_dates:
        selections_series = pd.Series(selections)
        portfolio_map = selections_series.reindex(backtest_months, method='ffill')

    returns_df_indexed = returns_df.set_index(['Trdmnt', 'Stkcd'])
    monthly_returns = []
    for month, stocks in portfolio_map.items():
        if not isinstance(stocks, set) or not stocks:
            monthly_returns.append(0.0)
            continue
        month_str = month.strftime('%Y-%m')
        try:
            relevant_returns = returns_df_indexed.loc[month_str]
            avg_return = relevant_returns[relevant_returns.index.isin(stocks)]['Mretwd'].mean()
            monthly_returns.append(avg_return if pd.notna(avg_return) else 0.0)
        except KeyError:
            monthly_returns.append(0.0)

    return pd.Series(monthly_returns, index=backtest_months, name=f"return_{strategy_name}")


def calculate_and_save_performance(
    all_returns_df: pd.DataFrame,
    benchmark_df: pd.DataFrame,
    risk_free_rate: float,
    output_dir: Path
):
    """
    计算并保存多个策略的详细绩效指标到CSV文件。
    """
    print("\n--- 各策略绩效对比报告 ---")
    # 确保输出目录存在
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / 'roic_groups_monthly_returns.csv'
    performance_output_file = output_dir / 'roic_groups_performance_summary.csv'

    results = all_returns_df.join(benchmark_df.set_index('date'), how='left').fillna(0)
    all_metrics = []
    total_months = len(results)

    annualized_benchmark_return = (1 + results['benchmark_return']).prod() ** (12 / total_months) - 1

    strategy_cols = [col for col in results.columns if col.startswith('return_')]
    for return_col in strategy_cols:
        strategy_name = return_col.replace('return_', '')
        cum_col = f'cum_{strategy_name}'
        results[cum_col] = (1 + results[return_col]).cumprod()

        final_return = results[cum_col].iloc[-1]
        annualized_return = final_return ** (12 / total_months) - 1
        annualized_volatility = results[return_col].std() * np.sqrt(12)
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0

        rolling_max = results[cum_col].expanding().max()
        drawdown = (results[cum_col] - rolling_max) / rolling_max
        max_drawdown = drawdown.min()

        excess_return = results[return_col] - results['benchmark_return']
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = excess_return.std() * np.sqrt(12)
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

        metrics = {
            '策略名称': strategy_name,
            '年化收益率': annualized_return, '年化波动率': annualized_volatility, '夏普比率': sharpe_ratio,
            '最大回撤': max_drawdown, '累计收益率': final_return - 1,
            '年化超额收益率': annualized_excess_return, '信息比率': information_ratio, '跟踪误差': tracking_error,
        }
        all_metrics.append(metrics)

    performance_df = pd.DataFrame(all_metrics).set_index('策略名称')
    performance_df.loc['基准 (沪深300)', '年化收益率'] = annualized_benchmark_return

    # 保存月度收益率文件
    results.to_csv(returns_output_file, encoding='utf-8-sig', float_format='%.6f')
    print(f"\n所有策略的月度收益数据已保存至: {returns_output_file}")

    # 格式化并保存绩效汇总文件
    formatted_df = performance_df.copy()
    percent_cols = ['年化收益率', '年化波动率', '最大回撤', '累计收益率', '年化超额收益率', '跟踪误差']
    for col in percent_cols:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) else '-')
    for col in ['夏普比率', '信息比率']:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"{x:.2f}" if pd.notna(x) else '-')

    formatted_df.to_csv(performance_output_file, encoding='utf-8-sig')
    print(f"所有策略的格式化绩效评估报告已保存至: {performance_output_file}")

    print("\n--- 绩效简报 ---")
    print(formatted_df.to_string())


# --- 步骤 2: 执行主流程 ---
try:
    # 1. 加载公用数据
    print("--- 正在加载公用数据 (股票收益率, 基准指数)... ---")
    returns_df = pd.read_csv(RETURNS_FILE)
    returns_df['Stkcd'] = returns_df['Stkcd'].astype(str).str.zfill(6)
    returns_df['Trdmnt'] = pd.to_datetime(returns_df['Trdmnt']).dt.strftime('%Y-%m')
    returns_df['Mretwd'] = pd.to_numeric(returns_df['Mretwd'], errors='coerce').fillna(0)

    all_benchmarks_df = pd.read_csv(BENCHMARK_FILE)
    benchmark_df = all_benchmarks_df[all_benchmarks_df['Indexcd'].astype(str).str.zfill(6) == BENCHMARK_CODE].copy()
    benchmark_df['date'] = pd.to_datetime(benchmark_df['Month'], format='%Y-%m')
    benchmark_df.rename(columns={'Idxrtn': 'benchmark_return'}, inplace=True)
    benchmark_df = benchmark_df[['date', 'benchmark_return']]
    print("数据加载完成。")

    # 2. 构建高、低ROIC投资组合
    print(f"\n--- 正在构建投资组合 (分位数阈值: {QUANTILE_THRESHOLD})... ---")
    high_roic_selections = build_portfolio_by_quantile(merged_df, 'roic', QUANTILE_THRESHOLD, select_top=True)
    low_roic_selections = build_portfolio_by_quantile(merged_df, 'roic', QUANTILE_THRESHOLD, select_top=False)
    print("投资组合构建完成。")
    print(f"高ROIC组共构建了 {len(high_roic_selections)} 个调仓期。")
    print(f"低ROIC组共构建了 {len(low_roic_selections)} 个调仓期。")

    # 3. 运行回测
    print("\n--- 正在运行回测... ---")
    high_roic_returns = run_single_strategy_backtest("High_ROIC", high_roic_selections, returns_df, START_DATE, END_DATE)
    low_roic_returns = run_single_strategy_backtest("Low_ROIC", low_roic_selections, returns_df, START_DATE, END_DATE)
    print("回测完成。")

    # 4. 合并收益并计算绩效，然后保存到文件
    all_returns_df = pd.concat([high_roic_returns, low_roic_returns], axis=1)
    calculate_and_save_performance(all_returns_df, benchmark_df, RISK_FREE_RATE, OUTPUT_DIR)

    print("\n--- 所有任务完成 ---")

except Exception as e:
    print(f"\n执行过程中出现严重错误: {e}")
    traceback.print_exc()

--- 正在加载公用数据 (股票收益率, 基准指数)... ---
数据加载完成。

--- 正在构建投资组合 (分位数阈值: 0.8)... ---
投资组合构建完成。
高ROIC组共构建了 179 个调仓期。
低ROIC组共构建了 179 个调仓期。

--- 正在运行回测... ---
回测完成。

--- 各策略绩效对比报告 ---

所有策略的月度收益数据已保存至: E:\PBROE\ch7\backtest_results\roic_groups_monthly_returns.csv
所有策略的格式化绩效评估报告已保存至: E:\PBROE\ch7\backtest_results\roic_groups_performance_summary.csv

--- 绩效简报 ---
             年化收益率   年化波动率  夏普比率     最大回撤     累计收益率 年化超额收益率  信息比率    跟踪误差
策略名称                                                                     
High_ROIC   20.55%  27.79%  0.63  -33.88%  1550.09%  19.16%  1.03  18.66%
Low_ROIC    20.84%  27.88%  0.64  -37.81%  1611.41%  19.46%  1.07  18.16%
基准 (沪深300)   1.39%       -     -        -         -       -     -       -

--- 所有任务完成 ---


In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

# =================================================================== #
#                           【1. 配置区域】                           #
# =================================================================== #
# 假设 dfpbroech7 变量已在之前的单元格中成功创建

# --- 输出文件配置 (可选，如果需要保存结果) ---
OUTPUT_DIR = Path(r'E:\PBROE\ch7\analysis_results')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
PARAMS_FILENAME = 'annual_regression_parameters.csv'


# =================================================================== #
#                       【2. 数据清洗与预处理】                       #
# =================================================================== #
print("--- 步骤 1: 数据清洗与预处理 ---")

# 确保 dfpbroech7 存在
if 'dfpbroech7' not in locals():
    print("错误: 变量 'dfpbroech7' 未定义。请先运行数据加载的代码单元格。")
    # 为了能独立运行，可以添加加载代码
    # dfpbroech7 = pd.read_csv('path_to_your_data.csv')
    df_clean = pd.DataFrame() # 创建一个空的DataFrame以避免后续代码报错
else:
    df = dfpbroech7.copy()

    # 1. 剔除ST,*ST股票 (假设 if_st=1 为ST)
    df = df[df['if_st'] != 1]

    # 2. 【回归模型特定清洗】剔除不符合回归条件的样本
    df = df[(df['PB'] > 0) &
            (df['roe_ttm'] <= 0.5) & (df['roe_ttm'] >= -0.1) &
            (df['roic'] <= 0.5) & (df['roic'] >= -0.1)]

    # 3. 剔除关键指标为空的记录
    df.dropna(subset=['roe_ttm', 'roic', 'PB', 'trdmnt'], inplace=True)

    # 4. 计算因变量和自变量
    df['lnPB'] = np.log(df['PB'])
    df['leverage_spread'] = df['roe_ttm'] - df['roic']

    # 5. 创建年份列用于年度回归
    df['year'] = pd.to_datetime(df['trdmnt'], format='%Y-%m').dt.year

    df_clean = df
    print(f"数据清洗后，剩余 {len(df_clean)} 条有效记录用于回归分析。")


# =================================================================== #
#                       【3. 全样本回归分析】                         #
# =================================================================== #
if not df_clean.empty:
    print("\n\n" + "="*80)
    print("【全样本回归分析】: ln(P/B) ~ 1 + roic + leverage_spread")
    print("="*80)

    try:
        # 准备 Y 和 X
        y_full = df_clean['lnPB']
        X_full = df_clean[['roic', 'leverage_spread']]
        X_full = sm.add_constant(X_full)

        # 执行OLS回归
        model_full = sm.OLS(y_full, X_full, missing='drop').fit()

        # 打印详细的回归结果摘要
        print(model_full.summary())

    except Exception as e:
        print(f"全样本回归失败: {e}")


# =================================================================== #
#                       【4. 年度滚动回归分析】                       #
# =================================================================== #
if not df_clean.empty:
    print("\n\n" + "="*80)
    print("【年度滚动回归分析】")
    print("="*80)

    annual_results = []

    # 按年份分组
    for year, group in df_clean.groupby('year'):
        # 样本量过少则跳过
        if len(group) < 100:
            print(f"年份 {year}: 样本量过少 ({len(group)}), 已跳过。")
            continue

        try:
            # 准备当年的 Y 和 X
            y_year = group['lnPB']
            X_year = group[['roic', 'leverage_spread']]
            X_year = sm.add_constant(X_year)

            # 执行OLS回归
            model_year = sm.OLS(y_year, X_year, missing='drop').fit()

            # 提取需要的参数
            params = {
                'year': year,
                'n_obs': model_year.nobs,
                'r_squared': model_year.rsquared_adj,
                'beta_0_const': model_year.params['const'],
                'p_val_const': model_year.pvalues['const'],
                'beta_1_roic': model_year.params['roic'],
                'p_val_roic': model_year.pvalues['roic'],
                'beta_2_leverage': model_year.params['leverage_spread'],
                'p_val_leverage': model_year.pvalues['leverage_spread']
            }
            annual_results.append(params)
            print(f"年份 {year}: 回归计算完成。")

        except Exception as e:
            print(f"年份 {year}: 回归失败 - {e}")

    # 将年度结果整理成DataFrame
    if annual_results:
        annual_params_df = pd.DataFrame(annual_results).set_index('year')

        print("\n--- 年度回归参数汇总表 ---")
        # 使用 display 函数在Jupyter中获得更好的表格显示效果
        try:
            from IPython.display import display
            display(annual_params_df.style.format({
                'n_obs': '{:,.0f}',
                'r_squared': '{:.3f}',
                'beta_0_const': '{:.3f}',
                'p_val_const': '{:.3f}',
                'beta_1_roic': '{:.3f}',
                'p_val_roic': '{:.3f}',
                'beta_2_leverage': '{:.3f}',
                'p_val_leverage': '{:.3f}'
            }))
        except (ImportError, NameError):
            print(annual_params_df.to_string(float_format="%.3f"))

        # 保存到CSV文件
        output_path = OUTPUT_DIR / PARAMS_FILENAME
        annual_params_df.to_csv(output_path, encoding='utf-8-sig', float_format='%.6f')
        print(f"\n年度回归参数已保存至: '{output_path}'")

    else:
        print("未能计算出任何年度回归结果。")



--- 步骤 1: 数据清洗与预处理 ---
数据清洗后，剩余 582202 条有效记录用于回归分析。


【全样本回归分析】: ln(P/B) ~ 1 + roic + leverage_spread
                            OLS Regression Results                            
Dep. Variable:                   lnPB   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                 1.393e+04
Date:                Fri, 25 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:19:13   Log-Likelihood:            -6.5470e+05
No. Observations:              582202   AIC:                         1.309e+06
Df Residuals:                  582199   BIC:                         1.309e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------

,n_obs,r_squared,beta_0_const,p_val_const,beta_1_roic,p_val_roic,beta_2_leverage,p_val_leverage
year,,,,,,,,
2005,"9,726",0.060,-0.475,0.000,1.109,0.000,2.413,0.000
2006,"12,894",0.073,-0.126,0.000,3.962,0.000,1.694,0.000
2007,"13,396",0.070,0.761,0.000,2.577,0.000,2.101,0.000
2008,"14,718",0.057,0.411,0.000,4.911,0.000,-0.477,0.000
2009,"15,327",0.018,0.901,0.000,0.743,0.000,1.036,0.000
2010,"16,483",0.021,1.091,0.000,2.249,0.000,0.290,0.000
2011,"20,075",0.021,0.886,0.000,2.183,0.000,0.425,0.000
2012,"23,779",0.052,0.458,0.000,3.397,0.000,0.716,0.000
2013,"26,568",0.035,0.525,0.000,2.846,0.000,1.105,0.000



年度回归参数已保存至: 'E:\PBROE\ch7\analysis_results\annual_regression_parameters.csv'


In [18]:
# 计算残差
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

# =================================================================== #
#                           【1. 配置区域】                           #
# =================================================================== #
# 假设 dfpbroech7 变量已在之前的单元格中成功创建

# --- 输出文件配置 ---
OUTPUT_DIR = Path(r'E:\PBROE\ch7')
OUTPUT_FILENAME = 'pbroe7.1Res.csv'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # 确保输出目录存在


# =================================================================== #
#                       【2. 数据清洗与预处理】                       #
# =================================================================== #
print("--- 步骤 1: 数据清洗与预处理 ---")

# 确保 dfpbroech7 存在
if 'dfpbroech7' not in locals():
    print("错误: 变量 'dfpbroech7' 未定义。请先运行数据加载的代码单元格。")
else:
    df = dfpbroech7.copy()

    # 1. 剔除ST,*ST股票 (假设 if_st=1 为ST)
    df = df[df['if_st'] != 1]

    # 2. 【回归模型特定清洗】剔除不符合回归条件的样本
    # PB必须大于0才能取对数；ROE和ROIC不能过高/过低以避免极端值影响
    df = df[(df['PB'] > 0) &
            (df['roe_ttm'] <= 0.5) & (df['roe_ttm'] >= -0.1) &
            (df['roic'] <= 0.5) & (df['roic'] >= -0.1)]

    # 3. 剔除关键指标为空的记录
    df.dropna(subset=['roe_ttm', 'roic', 'PB', 'indnme1'], inplace=True)

    # 4. 计算因变量和自变量
    df['lnPB'] = np.log(df['PB'])
    # 新模型的第二个自变量：杠杆贡献度
    df['leverage_spread'] = df['roe_ttm'] - df['roic']

    print(f"数据清洗后，剩余 {len(df)} 条有效记录用于回归分析。")


    # =================================================================== #
    #              【3. 执行月度行业内回归与残差计算】                    #
    # =================================================================== #
    print("\n--- 步骤 2: 执行基于ROIC分解模型的新回归与残差计算 ---")

    def calculate_residuals_roic_model(group):
        """
        对给定的分组（某行业某月份），执行新的多元回归并计算标准化残差。
        模型: ln(P/B) ~ 1 + roic + (roe_ttm - roic)
        """
        # 行业内样本数过少，无法进行有意义的回归
        if len(group) < 15: # 多元回归需要更多样本
            group['residual_zscore'] = np.nan
            return group

        try:
            # 准备回归的Y和X
            # Y是市净率的自然对数
            y = group['lnPB']
            # X包含常数项、ROIC和杠杆贡献度
            X = group[['roic', 'leverage_spread']]
            X = sm.add_constant(X) # 添加常数项

            # 执行OLS回归
            model = sm.OLS(y, X, missing='drop').fit()

            # 获取残差
            residuals = model.resid

            # 计算残差的均值和标准差
            resid_mean = residuals.mean()
            resid_std = residuals.std()

            # 标准化残差 (Z-score)
            if resid_std > 1e-6: # 避免除以零
                group['residual_zscore'] = (residuals - resid_mean) / resid_std
            else:
                group['residual_zscore'] = 0.0

        except Exception as e:
            # 如果回归出错，则将残差设为空值
            group['residual_zscore'] = np.nan

        return group

    # 按月份和行业分组，应用新的回归函数
    regression_results_df = df.groupby(['trdmnt', 'indnme1'], group_keys=False).apply(calculate_residuals_roic_model)

    # 清除回归失败或无法计算残差的记录
    regression_results_df.dropna(subset=['residual_zscore'], inplace=True)

    print(f"回归与残差计算完成，共得到 {len(regression_results_df)} 条有效记录。")


    # =================================================================== #
    #                       【4. 格式化并保存结果】                       #
    # =================================================================== #
    print("\n--- 步骤 3: 格式化并保存最终数据文件 ---")

    if not regression_results_df.empty:
        # 在这里，我们直接使用回归结果的DataFrame，它已经包含了所有需要的列
        final_data = regression_results_df.copy()

        # 【已修复】使用正确的日期格式 '%Y-%m' 来解析 trdmnt 列
        final_data['调入日期'] = pd.to_datetime(final_data['trdmnt'], format='%Y-%m') + pd.DateOffset(months=1)
        final_data['调入日期'] = final_data['调入日期'].dt.strftime('%Y-%m-%d')


        # 选择最终需要的列
        output_columns = ['调入日期', 'stkcd', 'shortname', 'indnme1', 'roe_ttm', 'roic', 'PB', 'residual_zscore']
        final_portfolio_data = final_data[output_columns]

        # 保存到CSV文件
        output_path = OUTPUT_DIR / OUTPUT_FILENAME
        final_portfolio_data.to_csv(output_path, index=False, encoding='utf-8-sig', float_format='%.6f')

        print(f"\n策略数据已成功生成并保存为 '{output_path}'。")
        print("该文件包含了基于新模型的标准化残差，可用于后续策略构建。")
    else:
        print("\n未能计算出任何有效残差，无法生成数据文件。")



--- 步骤 1: 数据清洗与预处理 ---
数据清洗后，剩余 582202 条有效记录用于回归分析。

--- 步骤 2: 执行基于ROIC分解模型的新回归与残差计算 ---


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3440\749038853.py:95: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regression_results_df = df.groupby(['trdmnt', 'indnme1'], group_keys=False).apply(calculate_residuals_roic_model)


回归与残差计算完成，共得到 528228 条有效记录。

--- 步骤 3: 格式化并保存最终数据文件 ---

策略数据已成功生成并保存为 'E:\PBROE\ch7\pbroe7.1Res.csv'。
该文件包含了基于新模型的标准化残差，可用于后续策略构建。

文件预览（按残差升序排列）：


,调入日期,stkcd,shortname,indnme1,roe_ttm,roic,PB,residual_zscore
18150,2024-10-01,000413,东旭光电,计算机、通信和其他电子设备制造业,-0.071403,-0.003696,0.096286,-6.201155
18149,2024-09-01,000413,东旭光电,计算机、通信和其他电子设备制造业,-0.077396,-0.005870,0.096286,-5.514598
428544,2024-07-01,600277,亿利洁能,化学原料和化学制品制造业,-0.035919,0.000200,0.071005,-5.027637
18148,2024-08-01,000413,东旭光电,计算机、通信和其他电子设备制造业,-0.077396,-0.005870,0.158742,-4.598220
428543,2024-06-01,600277,亿利洁能,化学原料和化学制品制造业,-0.030474,-0.006384,0.142010,-4.216615


In [27]:
# pbroe7_backtest_engine.py
# 一个为PB-ROE系列策略设计的、支持分组回测的通用引擎
# 版本：全向量化单进程版 (高性能、高稳定性)

import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import time

warnings.filterwarnings('ignore')

# =================================================================== #
#                       【1. 核心回测模块】                         #
# =================================================================== #

def run_grouped_backtest_vectorized(config):
    """
    (核心函数) 使用全向量化方法，对所有分组一次性完成回测。
    """
    print("--- 步骤 1: 加载数据 ---")
    try:
        # 加载策略分组数据
        strategy_df = pd.read_csv(config['RESIDUAL_FILE'])
        strategy_df['调入日期'] = pd.to_datetime(strategy_df['调入日期'])
        strategy_df['stkcd'] = strategy_df['stkcd'].astype(str).str.zfill(6)

        # 加载收益率数据
        returns_df = pd.read_csv(config['RETURNS_FILE'], dtype={'Stkcd': str})
        returns_df.rename(columns={'Stkcd': 'stkcd', 'Trdmnt': 'date', 'Mretwd': 'stock_return'}, inplace=True)
        returns_df['date'] = pd.to_datetime(returns_df['date'], format='%Y-%m')
        returns_df['stock_return'] = pd.to_numeric(returns_df['stock_return'], errors='coerce')

        # 加载基准数据
        all_benchmarks_df = pd.read_csv(config['BENCHMARK_FILE'], dtype={'Indexcd': str})
        benchmark_df = all_benchmarks_df[all_benchmarks_df['Indexcd'].str.zfill(6) == config['BENCHMARK_CODE']].copy()
        benchmark_df['date'] = pd.to_datetime(benchmark_df['Month'], format='%Y-%m')
        benchmark_df.rename(columns={'Idxrtn': 'benchmark_return'}, inplace=True)
        benchmark_df = benchmark_df[['date', 'benchmark_return']]
        print("所有数据加载成功。")

    except FileNotFoundError as e:
        print(f"错误: 找不到数据文件 {e.filename}。程序终止。")
        return None, None

    print("\n--- 步骤 2: 构建投资组合并执行回测 (向量化) ---")

    # 1. 创建分组
    num_groups = config['NUM_GROUPS']
    strategy_df['residual_group'] = strategy_df.groupby('调入日期')['residual_zscore'].transform(
        lambda x: pd.qcut(x, num_groups, labels=False, duplicates='drop') + 1
    )
    strategy_df.dropna(subset=['residual_group'], inplace=True)
    strategy_df['residual_group'] = strategy_df['residual_group'].astype(int)
    strategy_df.rename(columns={'调入日期': 'date'}, inplace=True)

    # 2. 筛选回测周期
    start_date = pd.to_datetime(config['BACKTEST_START_DATE'])
    end_date = pd.to_datetime(config['BACKTEST_END_DATE'])
    strategy_df = strategy_df[(strategy_df['date'] >= start_date) & (strategy_df['date'] <= end_date)]

    # 3. 【核心向量化步骤】将策略数据与收益数据合并
    # inner merge确保只保留在持仓期内有收益数据的股票
    merged_df = pd.merge(strategy_df[['date', 'stkcd', 'residual_group']], returns_df, on=['date', 'stkcd'], how='inner')

    # 4. 【核心向量化步骤】一次性计算所有组在所有月份的平均收益
    # groupby + mean 是Pandas中最高效的操作之一
    monthly_returns = merged_df.groupby(['date', 'residual_group'])['stock_return'].mean()

    # 5. 【核心向量化步骤】将结果从长格式转换为宽格式
    # unstack() 将 'residual_group' 索引级别变成列
    portfolio_returns_df = monthly_returns.unstack(level='residual_group')

    # 重命名列以匹配后续格式
    portfolio_returns_df.columns = [f'portfolio_return_g{int(col)}' for col in portfolio_returns_df.columns]

    # 填充缺失月份（即当月该组无持仓或无收益数据），确保时间序列连续
    all_months_index = pd.date_range(start=start_date, end=end_date, freq='MS')
    portfolio_returns_df = portfolio_returns_df.reindex(all_months_index, fill_value=0.0)

    print(f"向量化回测完成，已生成 {len(portfolio_returns_df)} 条月度收益记录。\n")

    return portfolio_returns_df, benchmark_df


# =================================================================== #
#                       【2. 绩效计算与保存】                         #
# =================================================================== #

def calculate_performance_and_save(portfolio_returns_df, benchmark_df, config):
    """为所有分组计算绩效并保存结果。"""
    print("--- 步骤 3: 计算并保存所有分组的绩效 ---")

    output_dir = config['OUTPUT_DIR']
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / f"{config['STRATEGY_NAME']}_returns.csv"
    performance_output_file = output_dir / f"{config['STRATEGY_NAME']}_performance.csv"

    num_groups = config['NUM_GROUPS']
    risk_free_rate = config['RISK_FREE_RATE']

    all_metrics = []
    # 合并基准收益
    final_returns_df = portfolio_returns_df.join(benchmark_df.set_index('date'), how='left').fillna(0)

    for group_id in range(1, num_groups + 1):
        return_col = f'portfolio_return_g{group_id}'
        if return_col not in final_returns_df.columns: continue

        final_returns_df[f'cumulative_return_g{group_id}'] = (1 + final_returns_df[return_col]).cumprod()
        total_months = len(final_returns_df)
        final_cumulative_return = final_returns_df[f'cumulative_return_g{group_id}'].iloc[-1]
        annualized_return = final_cumulative_return ** (12 / total_months) - 1
        annualized_volatility = final_returns_df[return_col].std() * np.sqrt(12)
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0
        rolling_max = final_returns_df[f'cumulative_return_g{group_id}'].expanding().max()
        drawdown = (final_returns_df[f'cumulative_return_g{group_id}'] - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
        annualized_benchmark_return = (1 + final_returns_df['benchmark_return']).prod() ** (12 / total_months) - 1
        excess_return = final_returns_df[return_col] - final_returns_df['benchmark_return']
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = excess_return.std() * np.sqrt(12)
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0
        metrics = {'group': f"Group {group_id}", '年化收益率': annualized_return, '年化波动率': annualized_volatility, '夏普比率': sharpe_ratio, '最大回撤': max_drawdown, '累计收益率': final_cumulative_return - 1, '年化超额收益率': annualized_excess_return, '信息比率': information_ratio, '跟踪误差': tracking_error}
        all_metrics.append(metrics)

    performance_df = pd.DataFrame(all_metrics)
    performance_df.loc['基准 (沪深300)', '年化收益率'] = annualized_benchmark_return

    # 整理要保存的列
    final_returns_df.reset_index(inplace=True)
    final_returns_df.rename(columns={'index': 'date'}, inplace=True)
    cols_to_save = ['date', 'benchmark_return'] + [f'portfolio_return_g{i}' for i in range(1, num_groups + 1) if f'portfolio_return_g{i}' in final_returns_df.columns] + [f'cumulative_return_g{i}' for i in range(1, num_groups + 1) if f'cumulative_return_g{i}' in final_returns_df.columns]
    final_returns_df_to_save = final_returns_df[cols_to_save]

    final_returns_df_to_save.to_csv(returns_output_file, index=False, encoding='utf-8-sig', float_format='%.6f')
    print(f"\n所有分组的月度收益率详情已保存至: {returns_output_file}")
    performance_df.to_csv(performance_output_file, index=False, encoding='utf-8-sig', float_format='%.6f')
    print(f"所有分组的绩效指标已保存至: {performance_output_file}")
    print(f"\n--- {config['STRATEGY_NAME']} 各分组绩效简报 ---")
    print(performance_df.to_string())


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main(config):
    """主执行函数"""
    start_time = time.time()

    # 步骤 1 & 2: 执行全向量化回测
    portfolio_returns_df, benchmark_df = run_grouped_backtest_vectorized(config)

    if portfolio_returns_df is None:
        print("回测失败，程序终止。")
        return

    # 步骤 3: 计算并保存绩效
    calculate_performance_and_save(portfolio_returns_df, benchmark_df, config)

    end_time = time.time()
    print(f"\n--- 所有任务完成！总耗时: {end_time - start_time:.2f} 秒 ---")

# =================================================================== #
#                       【4. 脚本执行入口】                              #
# =================================================================== #

if __name__ == "__main__":

    # --- 定义 pbroe7.1 分组回测的配置 ---
    CONFIG_PBROE7_GROUPED = {
        # --- 策略与输出配置 ---
        "STRATEGY_NAME": "pbroe7.1_grouped_backtest_vectorized",
        "OUTPUT_DIR": Path("E:/PBROE/ch7/backtest_results"),

        # --- 输入文件路径 ---
        "RESIDUAL_FILE": Path("E:/PBROE/ch7/pbroe7.1Res.csv"),
        "RETURNS_FILE": Path("E:/PBROE/data/TRDNEW_Mnth.csv"),
        "BENCHMARK_FILE": Path("E:/PBROE/data/benchmark_indices.csv"),

        # --- 策略核心参数 ---
        "NUM_GROUPS": 10,

        # --- 通用回测参数 ---
        "BACKTEST_START_DATE": '2010-05-01',
        "BACKTEST_END_DATE": '2025-04-30',
        "BENCHMARK_CODE": '000300',
        "RISK_FREE_RATE": 0.03
    }

    # --- 执行回测 ---
    # 调用主函数，并传入配置字典
    main(CONFIG_PBROE7_GROUPED)


--- 步骤 1: 加载数据 ---
所有数据加载成功。

--- 步骤 2: 构建投资组合并执行回测 (向量化) ---
向量化回测完成，已生成 180 条月度收益记录。

--- 步骤 3: 计算并保存所有分组的绩效 ---

所有分组的月度收益率详情已保存至: E:\PBROE\ch7\backtest_results\pbroe7.1_grouped_backtest_vectorized_returns.csv
所有分组的绩效指标已保存至: E:\PBROE\ch7\backtest_results\pbroe7.1_grouped_backtest_vectorized_performance.csv

--- pbroe7.1_grouped_backtest_vectorized 各分组绩效简报 ---
               group     年化收益率     年化波动率      夏普比率      最大回撤     累计收益率   年化超额收益率      信息比率      跟踪误差
0            Group 1  0.149185  0.253464  0.470224 -0.330524  7.050988  0.135329  0.915515  0.147818
1            Group 2  0.137539  0.256172  0.419791 -0.365079  5.910258  0.123683  0.748992  0.165133
2            Group 3  0.130207  0.261433  0.383300 -0.363842  5.271505  0.116352  0.664933  0.174983
3            Group 4  0.108281  0.269247  0.290741 -0.446471  3.674642  0.094425  0.510414  0.184998
4            Group 5  0.103705  0.276182  0.266872 -0.465326  3.393355  0.089850  0.469865  0.191224
5            Group 6  0.08352

In [ ]:
#计算残差时序分位数


